In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timedelta, date
import yaml
import requests
import os
import time

In [2]:
# Relative path to the root-level file
config_path = os.path.join("..", "..", "..", "config.yml")
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

In [3]:
# Read in municipality data
raw_data_filepath = config['raw_data']
municipality_filepath = raw_data_filepath+"Weather/"+"Municipal Boundaries of Portugal.csv"
municipalities = pd.read_csv(municipality_filepath, index_col=None)
municipalities = municipalities.round(2)
municipalities.tail()

,Name,Latitude,Longitude
301,Vouzela,40.67,-8.14
302,Águeda,40.59,-8.40
303,Évora,38.53,-7.87
304,Ílhavo,40.61,-8.70
305,Óbidos,39.36,-9.19


In [47]:
# # Loop through each of the municipalities and get weather data for each location

# # Setup the Open-Meteo API client with cache and retry on error
# cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
# retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
# openmeteo = openmeteo_requests.Client(session = retry_session)

# # Start and end date
# start_date = "2016-01-01"
# end_date = datetime.today().strftime('%Y-%m-%d')
# # end_date = "2010-03-01"

# # Create a list to store all processed DataFrames
# daily_dataframes = []

# # Iterate through each row in the municipalities DataFrame
# for index, row in municipalities.iterrows():
#     lat = row['Latitude']  # Extract latitude
#     long = row['Longitude']  # Extract longitude
#     municipality_name = row['Name']  # Extract municipality name (adjust the column name as needed)
#     time.sleep(10)  # Add a 1-second delay between requests

#     # Define API parameters
#     url = "https://archive-api.open-meteo.com/v1/archive"
#     params = {
#     	"latitude": lat,
#     	"longitude": long,
#     	"start_date": start_date,
#     	"end_date": end_date,
#     	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]
#     }

#     try:
#         # Make the API request
#         responses = openmeteo.weather_api(url, params=params)
#         response = responses[0]
        
#          # Process daily data. The order of variables needs to be the same as requested.
#         daily = response.Daily()
#         daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
#         daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
#         daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
        
#         daily_data = {"date": pd.date_range(
#         	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
#         	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
#         	freq = pd.Timedelta(seconds = daily.Interval()),
#         	inclusive = "left"
#         )}
#         daily_data["temperature_2m_max"] = daily_temperature_2m_max
#         daily_data["temperature_2m_min"] = daily_temperature_2m_min
#         daily_data["precipitation_sum"] = daily_precipitation_sum
#         daily_data["municipality"] = municipality_name
#         daily_data["latitude"] = lat
#         daily_data["longitude"] = long
        
#         daily_dataframe = pd.DataFrame(data = daily_data)
        
#         # Append the DataFrame to the list
#         daily_dataframes.append(daily_dataframe)

#         print(f"Processed municipality {municipality_name}")
    
#     except Exception as e:
#         print(f"Error processing municipality {municipality_name}: {e}")

# # Combine all DataFrames into one
# all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

Processed municipality Abrantes
Processed municipality Aguiar da Beira
Processed municipality Alandroal
Processed municipality Albergaria-a-Velha
Processed municipality Albufeira
Processed municipality Alcanena
Processed municipality Alcobaça
Processed municipality Alcochete
Processed municipality Alcoutim
Processed municipality Alcácer do Sal
Processed municipality Alenquer
Processed municipality Alfândega da Fé
Processed municipality Alijó
Processed municipality Aljezur
Processed municipality Aljustrel
Processed municipality Almada
Processed municipality Almeida
Processed municipality Almeirim
Processed municipality Almodôvar
Processed municipality Alpiarça
Processed municipality Alter do Chão
Processed municipality Alvaiázere
Processed municipality Alvito
Processed municipality Amadora
Processed municipality Amarante
Processed municipality Amares
Processed municipality Anadia
Processed municipality Angra do Heroísmo
Processed municipality Ansião
Processed municipality Arcos de Valde

KeyboardInterrupt: 

In [49]:
all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

In [ ]:
unique_names_list = list(all_daily_data["municipality"].unique())

In [52]:
# Write to csv
processed_data = config['top_level']+"data/raw/Weather/20241228_partial_municipalities_data.csv"
all_daily_data.to_csv(processed_data, index=True)

In [63]:
# Read in partial data from today
raw_data_filepath = config['top_level']
partial_data_filepath = raw_data_filepath+"data/raw/Weather/20241228_partial_municipalities_data.csv"
partial_data = pd.read_csv(partial_data_filepath, index_col=None)

In [64]:
partial_data.head()

,Unnamed: 0,date,temperature_2m_max,temperature_2m_min,precipitation_sum,municipality,latitude,longitude
0,0,2016-01-01 00:00:00+00:00,15.816500,9.616501,10.2,Abrantes,39.43,-8.16
1,1,2016-01-02 00:00:00+00:00,13.966499,7.316500,0.1,Abrantes,39.43,-8.16
2,2,2016-01-03 00:00:00+00:00,16.216500,8.816500,3.5,Abrantes,39.43,-8.16
3,3,2016-01-04 00:00:00+00:00,16.016500,10.816500,9.6,Abrantes,39.43,-8.16
4,4,2016-01-05 00:00:00+00:00,10.566500,6.516500,4.7,Abrantes,39.43,-8.16


In [73]:
# Convert a column from df1 into a list
list_from_df1 = partial_data['municipality'].tolist()

# Create a new DataFrame with values from df2 that are NOT in the list
# This df should be used instead of municipalities
filtered_df = municipalities[~municipalities['Name'].isin(list_from_df1)]
# filtered_df.head()

,Name,Latitude,Longitude
69,Castro Marim,37.29,-7.52
70,Castro Verde,37.70,-8.03
71,Celorico da Beira,40.62,-7.38
72,Celorico de Basto,41.40,-8.04
73,Chamusca,39.27,-8.38


In [77]:
# # Loop through each of the municipalities and get weather data for each location

# # Setup the Open-Meteo API client with cache and retry on error
# cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
# retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
# openmeteo = openmeteo_requests.Client(session = retry_session)

# # Start and end date
# start_date = "2016-01-01"
# end_date = datetime.today().strftime('%Y-%m-%d')
# # end_date = "2010-03-01"

# # Create a list to store all processed DataFrames
# daily_dataframes = []

# # Iterate through each row in the municipalities DataFrame
# for index, row in filtered_df.iterrows():
#     lat = row['Latitude']  # Extract latitude
#     long = row['Longitude']  # Extract longitude
#     municipality_name = row['Name']  # Extract municipality name (adjust the column name as needed)
#     time.sleep(10)  # Add a 1-second delay between requests

#     # Define API parameters
#     url = "https://archive-api.open-meteo.com/v1/archive"
#     params = {
#     	"latitude": lat,
#     	"longitude": long,
#     	"start_date": start_date,
#     	"end_date": end_date,
#     	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]
#     }

#     try:
#         # Make the API request
#         responses = openmeteo.weather_api(url, params=params)
#         response = responses[0]
        
#          # Process daily data. The order of variables needs to be the same as requested.
#         daily = response.Daily()
#         daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
#         daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
#         daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
        
#         daily_data = {"date": pd.date_range(
#         	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
#         	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
#         	freq = pd.Timedelta(seconds = daily.Interval()),
#         	inclusive = "left"
#         )}
#         daily_data["temperature_2m_max"] = daily_temperature_2m_max
#         daily_data["temperature_2m_min"] = daily_temperature_2m_min
#         daily_data["precipitation_sum"] = daily_precipitation_sum
#         daily_data["municipality"] = municipality_name
#         daily_data["latitude"] = lat
#         daily_data["longitude"] = long
        
#         daily_dataframe = pd.DataFrame(data = daily_data)
        
#         # Append the DataFrame to the list
#         daily_dataframes.append(daily_dataframe)

#         print(f"Processed municipality {municipality_name}")
    
#     except Exception as e:
#         print(f"Error processing municipality {municipality_name}: {e}")

# # Combine all DataFrames into one
# all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

In [75]:
# Combine first partial df with next partial df
all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

In [76]:
# Write to csv
processed_data = config['top_level']+"data/raw/Weather/20241228_partial_municipalities_data1.csv"
all_daily_data.to_csv(processed_data, index=True)

In [80]:
# Merge the data that we have
# # Concatenate the two DataFrames
# Read in partial data from today
raw_data_filepath = config['top_level']
partial_data_filepath = raw_data_filepath+"data/raw/Weather/20241228_partial_municipalities_data.csv"
partial_data1 = pd.read_csv(partial_data_filepath, index_col=None)
second_partial_data_filepath = raw_data_filepath+"data/raw/Weather/20241228_partial_municipalities_data1.csv"
partial_data2 = pd.read_csv(second_partial_data_filepath, index_col=None)

combined_df = pd.concat([partial_data1, partial_data2], ignore_index=True)

# # Identify duplicates based on all columns
# duplicates = combined_df[combined_df.duplicated(keep=False)]

# # Remove duplicates (keeping only the first occurrence)
# unique_df = combined_df.drop_duplicates(keep='first')

,Unnamed: 0,date,temperature_2m_max,temperature_2m_min,precipitation_sum,municipality,latitude,longitude
0,0,2016-01-01 00:00:00+00:00,15.816500,9.616501,10.2,Abrantes,39.43,-8.16
1,1,2016-01-02 00:00:00+00:00,13.966499,7.316500,0.1,Abrantes,39.43,-8.16
2,2,2016-01-03 00:00:00+00:00,16.216500,8.816500,3.5,Abrantes,39.43,-8.16
3,3,2016-01-04 00:00:00+00:00,16.016500,10.816500,9.6,Abrantes,39.43,-8.16
4,4,2016-01-05 00:00:00+00:00,10.566500,6.516500,4.7,Abrantes,39.43,-8.16
...,...,...,...,...,...,...,...,...
459895,233230,2024-12-24 00:00:00+00:00,11.887000,1.387000,0.0,Melgaço,42.05,-8.21
459896,233231,2024-12-25 00:00:00+00:00,13.437000,1.487000,0.0,Melgaço,42.05,-8.21
459897,233232,2024-12-26 00:00:00+00:00,12.937000,3.337000,0.0,Melgaço,42.05,-8.21
459898,233233,2024-12-27 00:00:00+00:00,7.037000,7.037000,NaN,Melgaço,42.05,-8.21


In [81]:
# Write to csv
processed_data = config['top_level']+"data/raw/Weather/20241228_partial_municipalities_data_combined.csv"
combined_df.to_csv(processed_data, index=True)

In [14]:
# Read in existing data - 12/29/2024
raw_data_filepath = config['top_level']
partial_data_filepath = raw_data_filepath+"data/raw/Weather/20241228_partial_municipalities_data_combined.csv"
partial_data = pd.read_csv(partial_data_filepath, index_col=None)

In [7]:
# Convert a column from df1 into a list
list_from_df = partial_data['municipality'].tolist()

# Create a new DataFrame with values from df2 that are NOT in the list
# This df should be used instead of municipalities
filtered_df = municipalities[~municipalities['Name'].isin(list_from_df)]
# filtered_df.head()

,Name,Latitude,Longitude
140,Mesão Frio,41.16,-7.87
141,Mira,40.43,-8.75
142,Miranda do Corvo,40.10,-8.33
143,Miranda do Douro,41.51,-6.36
144,Mirandela,41.51,-7.19


In [9]:
# Combine first partial df with next partial df
all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

In [10]:
# Write to csv
processed_data = config['top_level']+"data/raw/Weather/20241229_partial_municipalities_data.csv"
all_daily_data.to_csv(processed_data, index=True)

In [22]:
# Combine with existing data
combined_df = pd.concat([partial_data, all_daily_data], ignore_index=True)

In [25]:
# Write to .csv
processed_data = config['top_level']+"data/raw/Weather/20241229_partial_municipalities_data_combined.csv"
combined_df.to_csv(processed_data, index=True)

In [26]:
# Read in partial data from today
raw_data_filepath = config['top_level']
partial_data_filepath = raw_data_filepath+"data/raw/Weather/20241229_partial_municipalities_data_combined.csv"
partial_data = pd.read_csv(partial_data_filepath, index_col=None)

In [30]:
# Convert a column from df1 into a list
list_from_df = partial_data['municipality'].tolist()

# Create a new DataFrame with values from df2 that are NOT in the list
# This df should be used instead of municipalities
filtered_df = municipalities[~municipalities['Name'].isin(list_from_df)]
# filtered_df.head()

,Name,Latitude,Longitude
212,Póvoa de Varzim,41.42,-8.72
213,Redondo,38.63,-7.60
214,Reguengos de Monsaraz,38.40,-7.48
215,Resende,41.08,-7.94
216,Ribeira Brava,32.71,-17.03


In [31]:
# # Loop through each of the municipalities and get weather data for each location

# # Setup the Open-Meteo API client with cache and retry on error
# cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
# retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
# openmeteo = openmeteo_requests.Client(session = retry_session)

# # Start and end date
# start_date = "2016-01-01"
# end_date = datetime.today().strftime('%Y-%m-%d')
# # end_date = "2010-03-01"

# # Create a list to store all processed DataFrames
# daily_dataframes = []

# # Iterate through each row in the municipalities DataFrame
# for index, row in filtered_df.iterrows():
#     lat = row['Latitude']  # Extract latitude
#     long = row['Longitude']  # Extract longitude
#     municipality_name = row['Name']  # Extract municipality name (adjust the column name as needed)
#     time.sleep(10)  # Add a 1-second delay between requests

#     # Define API parameters
#     url = "https://archive-api.open-meteo.com/v1/archive"
#     params = {
#     	"latitude": lat,
#     	"longitude": long,
#     	"start_date": start_date,
#     	"end_date": end_date,
#     	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]
#     }

#     try:
#         # Make the API request
#         responses = openmeteo.weather_api(url, params=params)
#         response = responses[0]
        
#          # Process daily data. The order of variables needs to be the same as requested.
#         daily = response.Daily()
#         daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
#         daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
#         daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
        
#         daily_data = {"date": pd.date_range(
#         	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
#         	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
#         	freq = pd.Timedelta(seconds = daily.Interval()),
#         	inclusive = "left"
#         )}
#         daily_data["temperature_2m_max"] = daily_temperature_2m_max
#         daily_data["temperature_2m_min"] = daily_temperature_2m_min
#         daily_data["precipitation_sum"] = daily_precipitation_sum
#         daily_data["municipality"] = municipality_name
#         daily_data["latitude"] = lat
#         daily_data["longitude"] = long
        
#         daily_dataframe = pd.DataFrame(data = daily_data)
        
#         # Append the DataFrame to the list
#         daily_dataframes.append(daily_dataframe)

#         print(f"Processed municipality {municipality_name}")
    
#     except Exception as e:
#         print(f"Error processing municipality {municipality_name}: {e}")

# # Combine all DataFrames into one
# all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

Processed municipality Póvoa de Varzim
Processed municipality Redondo
Processed municipality Reguengos de Monsaraz
Processed municipality Resende
Processed municipality Ribeira Brava
Processed municipality Ribeira Grande
Processed municipality Ribeira de Pena
Processed municipality Rio Maior
Processed municipality Sabrosa
Processed municipality Sabugal
Processed municipality Salvaterra de Magos
Processed municipality Santa Comba Dão
Processed municipality Santa Cruz
Processed municipality Santa Cruz da Graciosa
Processed municipality Santa Maria da Feira
Processed municipality Santa Marta de Penaguião
Processed municipality Santana
Processed municipality Santarém
Processed municipality Santiago do Cacém
Processed municipality Santo Tirso
Processed municipality Sardoal
Processed municipality Seia
Processed municipality Seixal
Processed municipality Sernancelhe
Processed municipality Serpa
Processed municipality Sertã
Processed municipality Sesimbra
Processed municipality Setúbal
Process

In [32]:
# Combine first partial df with next partial df
all_daily_data = pd.concat(daily_dataframes, ignore_index=True)

In [33]:
# Write to csv
processed_data = config['top_level']+"data/raw/Weather/20241229_partial_municipalities_data2.csv"
all_daily_data.to_csv(processed_data, index=True)

In [34]:
# Combine with existing data
combined_df = pd.concat([partial_data, all_daily_data], ignore_index=True)

In [36]:
combined_df.tail()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,date,temperature_2m_max,temperature_2m_min,precipitation_sum,municipality,latitude,longitude
936365,NaN,NaN,NaN,2024-12-25 00:00:00+00:00,15.743000,6.243,0.0,Vila Nova de Gaia,41.07,-8.58
936366,NaN,NaN,NaN,2024-12-26 00:00:00+00:00,16.193001,4.793,0.0,Vila Nova de Gaia,41.07,-8.58
936367,NaN,NaN,NaN,2024-12-27 00:00:00+00:00,15.043000,3.593,0.0,Vila Nova de Gaia,41.07,-8.58
936368,NaN,NaN,NaN,2024-12-28 00:00:00+00:00,4.143000,4.143,NaN,Vila Nova de Gaia,41.07,-8.58
936369,NaN,NaN,NaN,2024-12-29 00:00:00+00:00,NaN,NaN,NaN,Vila Nova de Gaia,41.07,-8.58


In [37]:
# Write to .csv
processed_data = config['top_level']+"data/raw/Weather/20241229_partial_municipalities_data_combined2.csv"
combined_df.to_csv(processed_data, index=True)